Imports Python

In [2]:
import pandas as pd
import numpy as np

Leitura dos datasets

In [3]:
df_regular = pd.read_csv('NBA/2021-2022 NBA Player Stats - Regular.csv', encoding='latin-1', sep=';')
df_playoffs = pd.read_csv('NBA/2021-2022 NBA Player Stats - Playoffs.csv', encoding='latin-1', sep=';')


Juntando os dois datasets

In [4]:
df_total = pd.concat([df_regular, df_playoffs], ignore_index=True)

In [5]:
df_total.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,...,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,...,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,...,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,...,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9


Iniciando Analise Exploratória dos Dados

In [6]:
df_total.describe()
#df_total.info()

,Rk,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.00000,1029.000000,1029.000000,1029.000000,1029.000000,...,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000
mean,263.273081,26.165209,30.801749,14.001944,18.510787,2.90690,6.460641,0.428614,0.884645,2.590087,...,0.650882,0.804665,2.542371,3.346939,1.812536,0.583188,0.355199,1.001263,1.610982,7.948397
std,175.235007,4.101807,25.821475,21.946566,10.417364,2.33211,4.931101,0.156789,0.870237,2.279284,...,0.297500,0.767213,1.884674,2.462849,1.874005,0.441873,0.381993,0.892461,0.915318,6.481502
min,1.000000,19.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,116.000000,23.000000,6.000000,0.000000,10.000000,1.20000,2.800000,0.378000,0.100000,0.700000,...,0.591000,0.300000,1.200000,1.600000,0.500000,0.300000,0.100000,0.400000,1.000000,3.100000
50%,227.000000,25.000000,22.000000,3.000000,17.600000,2.40000,5.100000,0.439000,0.700000,2.100000,...,0.750000,0.600000,2.300000,2.900000,1.100000,0.500000,0.300000,0.800000,1.600000,6.300000
75%,412.000000,29.000000,55.000000,17.000000,27.200000,4.00000,9.000000,0.500000,1.400000,4.000000,...,0.840000,1.100000,3.400000,4.500000,2.400000,0.900000,0.500000,1.300000,2.300000,10.900000
max,605.000000,41.000000,82.000000,82.000000,44.000000,12.20000,23.800000,1.000000,4.500000,11.700000,...,1.000000,5.500000,12.000000,14.700000,10.800000,2.500000,2.800000,6.200000,5.000000,31.700000
